# Calculate aggregated CFs for AWARE

Create a [conda](https://conda.io/miniconda.html) environment with the following:

    conda create -n df69 -c conda-forge -c cmutel -c haasad -c konstantinstadler python=3.6 bw2regional rower bw2_aware jupyter
    
Then activate it and run `jupyter notebook`.

In [66]:
from bw2regional.pandarus_remote import remote, AlreadyExists
import brightway2 as bw
import bw2_aware
import bw2regional as bwr
import os
import fiona
import numpy as np

In [2]:
assert bw2_aware.__version__ >= (0, 2)

In [3]:
bw.projects.set_current("AWARE Aggregated CFs")

In [6]:
bw.create_default_biosphere3()

Writing activities to SQLite3 database:
0%                          100%
[#################             ] | ETA: 00:00:00

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/29/2018 12:05:54
  Finished: 10/29/2018 12:05:54
  Total time elapsed: 00:00:00
  CPU %: 100.10
  Memory %: 0.51
Created database: biosphere3


In [8]:
bw.create_core_migrations()

# Install basic world definitions

In [9]:
bwr.bw2regionalsetup()

Creating `rower` 'RoW' geo/topocollections


# Import AWARE

In [13]:
bw2_aware.import_aware()

0%                          100%


Merging topographical faces for geocollection RoW


[##############################] | ETA: 00:00:00
Total time elapsed: 00:05:27


Creating intersection (RoW, world-topo-watersheds-aware)


0%                          100%
[#                             ] | ETA: 00:00:02

Merging topographical faces for geocollection ecoinvent


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:11
0%                          100%
[##                            ] | ETA: 00:00:02

Creating intersection (ecoinvent, world-topo-watersheds-aware)
Merging topographical faces for geocollection world


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:02


Creating intersection (world, world-topo-watersheds-aware)
Skipping existing intersection: (world-topo-watersheds-aware, watersheds-aware)


In [21]:
sorted(bw.methods)

[('AWARE', '1.2 (April 2017)', 'Annual', 'Agricultural'),
 ('AWARE', '1.2 (April 2017)', 'Annual', 'Non-agricultural')]

# Create fake database

Need a database with one activity for each region in ecoinvent and each country in the world

In [15]:
with fiona.open(bwr.geocollections['ecoinvent']['filepath']) as f:
    ecoinvent_locations = [
        ('ecoinvent', feat['properties'][bwr.geocollections['ecoinvent']['field']]) 
        for feat in f
    ]

In [16]:
with fiona.open(bwr.geocollections['world']['filepath']) as f:
    world_locations = [
        feat['properties'][bwr.geocollections['world']['field']]
        for feat in f
    ]

In [18]:
all_locations = [x for x in (ecoinvent_locations + world_locations) if x]

In [24]:
water = bw.Method(('AWARE', '1.2 (April 2017)', 'Annual', 'Agricultural')).load()[0][0]

('biosphere3', '478e8437-1c21-4032-8438-872a6b5ddcdf')

In [27]:
data = {
    ('empty', str(i)): {
        'location': location,
        'name': str(i),
        'type': 'process',
        'exchanges': [{
           'amount': 1.0,
           'input': water,
           'type': 'biosphere',
        }]
    }
    for i, location in enumerate(all_locations)
}

In [28]:
bw.Database("empty").write(data)

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/29/2018 14:59:25
  Finished: 10/29/2018 14:59:25
  Total time elapsed: 00:00:00
  CPU %: 104.80
  Memory %: 0.72


In [29]:
bw.databases['empty']['geocollections'] = ['world', 'ecoinvent']
bw.databases['biosphere3']['geocollections'] = []
bw.databases.flush()

# Add `loadings` extension table

In [30]:
fp = os.path.abspath("loadings/loadings.tif")
assert os.path.isfile(fp)

bwr.geocollections['loadings'] = {
    'filepath': fp,
    'band': 1
}

## Calculate raster statistics for our extension tables

In [32]:
try:
    remote.calculate_rasterstats('world-topo-watersheds-aware', 'loadings')
except AlreadyExists:
    pass

Calculation job submitted.


In [33]:
remote.rasterstats_as_xt('world-topo-watersheds-aware', 'loadings', "xt-loadings")

Brightway2 ExtensionTable: xt-loadings

# Non-agricultural CFs

In [35]:
fake_lca = bwr.ExtensionTablesLCA(
    {bw.Database("empty").random(): 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Non-agricultural'),
    xtable="xt-loadings"
)
fake_lca.lci()
fake_lca.lcia()

In [41]:
fake_lca.fix_spatial_dictionaries()

In [75]:
reg_cf_inv_scale = (
    fake_lca.distribution_normalization_matrix  *
    fake_lca.distribution_matrix                *
    fake_lca.xtable_matrix                      *
    fake_lca.geo_transform_normalization_matrix *
    fake_lca.geo_transform_matrix               *
    fake_lca.reg_cf_matrix
)

In [76]:
reg_cf_inv_scale

<448x1 sparse matrix of type '<class 'numpy.float64'>'
	with 418 stored elements in Compressed Sparse Row format>

In [77]:
aggregated_cfs = dict([
    (key, reg_cf_inv_scale[index, 0]) 
    for key, index in fake_lca.inv_spatial_dict.items()
])

In [78]:
aggregated_cfs

{('ecoinvent', 'AUS-ACT'): 88.19999092772075,
 ('ecoinvent', 'AUS-NSW'): 85.87650285669463,
 ('ecoinvent', 'AUS-NTR'): 4.176353907861551,
 ('ecoinvent', 'AUS-QNS'): 44.41609998356238,
 ('ecoinvent', 'AUS-SAS'): 53.71586266060618,
 ('ecoinvent', 'AUS-TSM'): 1.4544925344140922,
 ('ecoinvent', 'AUS-VCT'): 74.53226869853867,
 ('ecoinvent', 'AUS-WAS'): 10.814266996556817,
 ('ecoinvent', 'Asia without China'): 34.86497823921746,
 ('ecoinvent', 'BALTSO'): 1.198126889732204,
 ('ecoinvent', 'BR-AC'): 0.6191374882708339,
 ('ecoinvent', 'BR-AL'): 2.7985729879735772,
 ('ecoinvent', 'BR-AM'): 0.15068149487110497,
 ('ecoinvent', 'BR-AP'): 0.17171708784097073,
 ('ecoinvent', 'BR-BA'): 3.639659583571307,
 ('ecoinvent', 'BR-CE'): 13.200045915221223,
 ('ecoinvent', 'BR-DF'): 0.921846605275083,
 ('ecoinvent', 'BR-ES'): 2.0259999027655704,
 ('ecoinvent', 'BR-GO'): 0.8389536463117786,
 ('ecoinvent', 'BR-MA'): 2.529634073731055,
 ('ecoinvent', 'BR-MG'): 2.52900569916641,
 ('ecoinvent', 'BR-MS'): 1.112326102

# Agricultural CFs

In [79]:
fake_lca = bwr.ExtensionTablesLCA(
    {bw.Database("empty").random(): 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Agricultural'),
    xtable="xt-loadings"
)
fake_lca.lci()
fake_lca.lcia()

In [80]:
fake_lca.fix_spatial_dictionaries()

In [81]:
reg_cf_inv_scale = (
    fake_lca.distribution_normalization_matrix  *
    fake_lca.distribution_matrix                *
    fake_lca.xtable_matrix                      *
    fake_lca.geo_transform_normalization_matrix *
    fake_lca.geo_transform_matrix               *
    fake_lca.reg_cf_matrix
)

In [82]:
reg_cf_inv_scale

<448x1 sparse matrix of type '<class 'numpy.float64'>'
	with 395 stored elements in Compressed Sparse Row format>

In [83]:
aggregated_cfs = dict([
    (key, reg_cf_inv_scale[index, 0]) 
    for key, index in fake_lca.inv_spatial_dict.items()
])

In [84]:
aggregated_cfs

{('ecoinvent', 'AUS-ACT'): 91.9999937200908,
 ('ecoinvent', 'AUS-NSW'): 88.16839484189487,
 ('ecoinvent', 'AUS-NTR'): 3.807706283937677,
 ('ecoinvent', 'AUS-QNS'): 46.808991070463065,
 ('ecoinvent', 'AUS-SAS'): 72.86031929617457,
 ('ecoinvent', 'AUS-TSM'): 2.4220013314947737,
 ('ecoinvent', 'AUS-VCT'): 78.60312760774293,
 ('ecoinvent', 'AUS-WAS'): 13.331988779081266,
 ('ecoinvent', 'Asia without China'): 42.88358041897469,
 ('ecoinvent', 'BALTSO'): 1.5200150457166086,
 ('ecoinvent', 'BR-AC'): 0.7643090590019641,
 ('ecoinvent', 'BR-AL'): 2.454411553890886,
 ('ecoinvent', 'BR-AM'): 0.16277172994335215,
 ('ecoinvent', 'BR-AP'): 0.19106880795895553,
 ('ecoinvent', 'BR-BA'): 2.7145458548356944,
 ('ecoinvent', 'BR-CE'): 9.684001547323996,
 ('ecoinvent', 'BR-DF'): 0.7845744744372994,
 ('ecoinvent', 'BR-ES'): 0.9022492403848513,
 ('ecoinvent', 'BR-GO'): 0.7870423033810876,
 ('ecoinvent', 'BR-MA'): 2.733258663948445,
 ('ecoinvent', 'BR-MG'): 1.716724903601102,
 ('ecoinvent', 'BR-MS'): 0.9337997